análisis de fuerza bruta por frecuencias. 

In [1]:
import string
from collections import Counter
from itertools import product
import re

# Frecuencia de letras en español actualizada
FRECUENCIAS_ES = {
    'A': 11.525, 'B': 2.215, 'C': 4.019, 'D': 5.010, 'E': 12.181, 'F': 0.692,
    'G': 1.768, 'H': 0.703, 'I': 6.247, 'J': 0.493, 'K': 0.011, 'L': 4.967,
    'M': 3.157, 'N': 6.712, 'Ñ': 0.311, 'O': 8.683, 'P': 2.510, 'Q': 0.877,
    'R': 6.871, 'S': 7.977, 'T': 4.632, 'U': 2.927, 'V': 1.138, 'W': 0.017,
    'X': 0.215, 'Y': 1.008, 'Z': 0.467
}

ALFABETO = "ABCDEFGHIJKLMNÑOPQRSTUVWXYZ"

def clean_text(text):
    text = text.upper()
    text = re.sub(r'[^A-ZÑ]', '', text)  # Mantener solo letras
    return text

# Función de puntuación basada en frecuencias
def score_text(text):
    text = clean_text(text)
    contador = Counter(text)
    total_letras = sum(contador[letra] for letra in ALFABETO)
    score = sum((contador[letra] / total_letras) * FRECUENCIAS_ES.get(letra, 0) for letra in ALFABETO)
    return score



Cifrado Caesar

In [2]:
# Descifrado César
def decrypt_caesar(text, shift):
    decrypted = "".join(ALFABETO[(ALFABETO.index(c) - shift) % len(ALFABETO)] if c in ALFABETO else c for c in text)
    return decrypted



Cifrado afín

In [3]:
# Descifrado Afín
def mod_inverse(a, m=len(ALFABETO)):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

def decrypt_affine(text, a, b):
    a_inv = mod_inverse(a, len(ALFABETO))
    if a_inv is None:
        return None
    decrypted = "".join(ALFABETO[(a_inv * (ALFABETO.index(c) - b)) % len(ALFABETO)] if c in ALFABETO else c for c in text)
    return decrypted



Cifrado Vigenére

In [4]:
# Descifrado Vigenère
def decrypt_vigenere(text, key):
    key = clean_text(key)
    decrypted = "".join(ALFABETO[(ALFABETO.index(c) - ALFABETO.index(key[i % len(key)])) % len(ALFABETO)] if c in ALFABETO else c for i, c in enumerate(text))
    return decrypted


In [8]:
# Función de fuerza bruta
def brute_force_decrypt(text, cipher_type, k=5):
    text = clean_text(text)
    results = []
    
    if cipher_type == "caesar":
        for shift in range(31):
            decrypted = decrypt_caesar(text, shift)
            score = score_text(decrypted)
            results.append((decrypted, shift, score))
    
    elif cipher_type == "affine":
        for a, b in product(range(1, 17), range(1, 17)):
            decrypted = decrypt_affine(text, a, b)
            if decrypted:
                score = score_text(decrypted)
                results.append((decrypted, (a, b), score))
    
    elif cipher_type == "vigenere":
        common_keys = ["PA", "PAE", "PAO", "PAEL", "PAES", "PAQUE"]  # Claves probables
        for key in common_keys:
            decrypted = decrypt_vigenere(text, key)
            score = score_text(decrypted)
            results.append((decrypted, key, score))
    
    results.sort(key=lambda x: x[2], reverse=True)
    return results[:k]

# Ejemplo de uso
file_path = "ceasar.txt"
resultados = brute_force_decrypt(file_path, "caesar", k=3)
for res in resultados:
    print(res)

# Ejemplo de uso
file_path = "afines.txt"
resultados = brute_force_decrypt(file_path, "affine", k=3)
for res in resultados:
    print(res)

# Ejemplo de uso
file_path = "vigenere.txt"
resultados = brute_force_decrypt(file_path, "vigenere", k=3)
for res in resultados:
    print(res)


('NOLDLCEIE', 16, 7.218555555555555)
('CEASARTXT', 0, 7.06411111111111)
('CEASARTXT', 27, 7.06411111111111)
('LPSXODEIE', (1, 16), 6.663444444444444)
('OSPTCÑERE', (8, 15), 6.59611111111111)
('ZDAENYOCO', (8, 8), 6.476444444444444)
('GIQEXECEEXE', 'PA', 6.749636363636363)
('GICONACEPIT', 'PAE', 6.231181818181818)
('GIRONPCEFIT', 'PAO', 5.505636363636364)
